# LangChain Expression Language(LCEL)

- Author: [Erika](https://github.com/ErikaPark)
- Peer Review: [Wooseok Jeong](https://github.com/jeong-wooseok), [JeongGi Park](https://github.com/jeongkpa)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)


## Overview

**LangChain Expression Language (LCEL)** is a declarative expression language used within the LangChain framework to define and customize the behavior of chains and agents. It is designed as a flexible tool that enables users to simplify complex workflows and make better use of LangChain's capabilities in an intuitive manner.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)

### References


----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain-anthropic",
        "langchain_community",
        "langchain_text_splitters",
        "langchain_openai",
        "langchain-teddynote"
    ],
    verbose=False,
    upgrade=False,
)

In [5]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "LCEL",  
    }
)

Environment variables have been set successfully.


In [3]:
# Configuration file to manage the API KEY as an environment variable
from dotenv import load_dotenv

# Load API KEY information
load_dotenv()


True


##  Basic Example: Prompt + Model + Output Parser
The most fundamental and common use case is connecting a prompt template with a model. To demonstrate how this works, let's create a Chain that asks for the capital cities of various countries.

In [4]:
# Set up LangSmith tracking: https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# Enter the project name
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## Utilizing Prompt Templates

`PromptTemplate`

- A prompt template is a structured tool used to dynamically generate prompts based on user input data.
- Usage
  - `template`: A template string where curly braces `{}` represent variables.
  - `input_variables`: A list defining the names of variables to be substituted into the curly braces.

`input_variables`

- `input_variables` is a list that defines the names of the variables used in the `PromptTemplate`.

In [5]:
from langchain_teddynote.messages import stream_response  # Streaming Output
from langchain_core.prompts import PromptTemplate

Create a `PromptTemplate` object using the `from_template()` method.

In [5]:
# Define the template
template = "What is the capital of {country}?"

# Create a PromptTemplate object using the from_template method
prompt_template = PromptTemplate.from_template(template)
prompt_template


PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [6]:
# Generate the prompt
prompt = prompt_template.format(country="South Korea")
prompt

'What is the capital of South Korea?'

In [7]:
# Generate the prompt
prompt = prompt_template.format(country="United States")
prompt


'What is the capital of United States?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

## Creating a Chain

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

Here, we use LCEL to combine various components into a single chain:

```
chain = prompt | model | output_parser
```

The `|`` operator, similar to the [Unix pipe operator](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)connects different components, passing the output of one component as the input to the next.

In this chain:

1. User input is passed to the prompt template.
2. The prompt template's output is then sent to the model.
3. Each component can be examined individually to understand what is happening at each stage.

This modular design simplifies the process of connecting inputs, processing, and outputs into a streamlined workflow.


In [11]:
# Create the prompt as a PromptTemplate object
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")

# Initialize the model
model = ChatOpenAI()

# Combine the prompt and model into a chain
chain = prompt | model

### Calling `invoke()`
* Input values are passed as a Python dictionary (key: value format).
* The `invoke()` function is used to call the chain and process the input.

In [12]:
# Set the input dictionary with the topic 'How AI models learn'
input = {"topic": "How AI models learn"}

In [15]:
# prompt.format(**input)

'Please explain How AI models learn in simple terms.'

In [13]:
# Connect the prompt object and model object using the pipe (|) operator
# Use the invoke method to pass the input
# This returns the message generated by the AI model
response = chain.invoke(input)
response

AIMessage(content='AI models learn by being trained on large amounts of data. The data is fed into the model, which then uses algorithms to analyze and identify patterns and relationships within the data. As the model processes more data, it adjusts and refines its algorithms to improve its accuracy and performance. This process is known as machine learning, where the model learns from the data it is exposed to and becomes more proficient at making predictions or decisions based on that data. Over time, the model becomes more accurate and reliable as it continues to learn from new data and experiences.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 17, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'syste

Below is an example of streaming output.

In [18]:
# Request for streaming output
answer = chain.stream(input)
# Streaming output
stream_response(answer)

AI models learn through a process called machine learning, which involves feeding the model large amounts of data and allowing it to identify patterns and relationships within that data. 

The model uses algorithms to analyze the data and make predictions based on what it has learned. As the model is exposed to more data, it adjusts and refines its predictions, improving its accuracy over time.

In simple terms, AI models learn by being exposed to data, identifying patterns, and refining their predictions based on that data. The more data they are exposed to, the better they become at making accurate predictions.

### Output Parser

An **Output Parser** is a tool designed to transform or process the responses from an AI model into a specific format. Since the model's output is typically provided as free-form text, an **Output Parser** is essential to convert it into a structured format or extract the required data.

In [19]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser() # Directly returns the model's response as a string without modification.

Add an output parser to the chain.

In [21]:
# Combine the prompt, model, and output parser to create a processing chain
chain = prompt | model | output_parser

In [20]:
# Use the invoke method of the chain object to pass the input
input = {"topic": "How AI models learn"}
chain.invoke(input)

AIMessage(content='AI models learn by processing large amounts of data and adjusting their parameters to improve their performance in completing a specific task. The process can be broken down into the following steps:\n\n1. Data Collection: The AI model is trained on a large dataset that contains examples of input data and the corresponding correct output. This data is used to teach the model how to perform the task it is designed for.\n\n2. Training: The AI model processes the input data and makes predictions based on its current parameters. It compares these predictions to the correct output from the dataset and calculates how well it performed. The model then adjusts its parameters to minimize the error between its predictions and the correct output.\n\n3. Evaluation: The model is tested on a separate dataset to assess how well it has learned from the training data. This evaluation helps to determine if the model is performing well and if further adjustments are needed.\n\n4. Itera

In [22]:
# Request for streaming output
answer = chain.stream(input)
# Streaming output
stream_response(answer)

AI models learn by processing large amounts of data and using algorithms to identify patterns and relationships within that data. These models are trained on labeled data, which means that the data is already categorized or labeled with the correct answer. 

During the training process, the AI model adjusts its parameters and weights based on feedback it receives from the labeled data. This feedback helps the model to make more accurate predictions or classifications in the future. 

Through this iterative process of training and feedback, the AI model learns to recognize patterns and make predictions on new, unseen data. In essence, AI models learn by continuously refining their algorithms and improving their accuracy based on the data they are trained on.

### Apply Changes to the Template

- You can **modify** the prompt content below and test it as needed.
- You can also change the `model_name` for testing purposes.

In [27]:
template = """
You are a teacher with 10 years of experience teaching English. Please create an English conversation that fits the given situation.
Refer to the [FORMAT] to structure your response.

# Situation:
{question}

#FORMAT:
- Dialogue in English:
- Explanation of the Dialogue: 
"""

# Generate the prompt using the PromptTemplate
prompt = PromptTemplate.from_template(template)

# Initialize the ChatOpenAI model
model = ChatOpenAI(model_name="gpt-4-turbo")

# Initialize the string output parser
output_parser = StrOutputParser()


In [25]:
# Construct the chain
chain = prompt | model | output_parser

In [26]:
# Execute the completed chain to get the response
print(chain.invoke({"question": "I want to go to a restaurant and order food"}))

# Dialogue in English:
A: "Good evening. I’d like a table for two, please."

Host: "Certainly! Do you have a reservation?"

A: "No, we don’t. Is there a wait?"

Host: "Just a short one. It should be about 10 minutes. May I have your name, please?"

A: "It’s Alex."

Host: "Thank you, Alex. We’ll call you as soon as your table is ready."

[10 minutes later]

Host: "Alex, your table is ready. Please follow me."

A: "Thank you."

[Seated at the table]

Waiter: "Hello, I’m Jim and I’ll be your server today. Can I start you off with something to drink?"

A: "Yes, could we have two glasses of water, please? And also, could you bring us the wine list?"

Waiter: "Absolutely, I’ll be right back with your waters and the wine list."

[Waiter returns with drinks and the wine list]

Waiter: "Here you are. Are you ready to order, or do you need a few more minutes?"

A: "I think we’ll need a few more minutes. We’re still deciding."

Waiter: "No problem at all. I’ll come back in a few minutes. Please t

In [28]:
# Execute the completed chain to get the response
# Request for streaming output
answer = chain.stream({"question": "I want to go to a restaurant and order food"})
# Streaming output
stream_response(answer)


# Dialogue in English:
**Alex:** Good evening! Could I see the menu, please?

**Waiter:** Of course! Here you go. Our special today is grilled salmon with asparagus. Would you like to start with something to drink?

**Alex:** Yes, I'll have a glass of white wine, please. Could you recommend a wine that goes well with salmon?

**Waiter:** Certainly! The Chardonnay is excellent with our salmon dish. It’s light and complements the flavors beautifully.

**Alex:** Great, I'll have a glass of Chardonnay then. And for the meal, I'll try the special, the grilled salmon.

**Waiter:** Excellent choice! Would you like any sides or a salad with that?

**Alex:** What side dishes do you have?

**Waiter:** We have mashed potatoes, steamed vegetables, or a house salad.

**Alex:** I'll go with steamed vegetables, please.

**Waiter:** Perfect. Your order will be ready shortly. Enjoy your meal!

**Alex:** Thank you!

# Explanation:
This conversation is a typical interaction between a customer (Alex) and 

In [29]:
# This time, set the question to 'Ordering pizza in the United States' and execute it.
# Request for streaming output
answer = chain.stream({"question": "Ordering pizza in the United States"})
# Streaming output
stream_response(answer)

# Dialogue in English:
**Customer:** Hi, I’d like to order a pizza for delivery, please.

**Pizza Shop Employee:** Sure, what would you like to order today?

**Customer:** I’d like a large pepperoni pizza with extra cheese.

**Pizza Shop Employee:** Would you like to add any sides or drinks to your order?

**Customer:** Yes, can I get an order of garlic bread and two cans of Coke?

**Pizza Shop Employee:** Absolutely. Can I have your delivery address, please?

**Customer:** It's 452 Maple Street, Apartment 2B.

**Pizza Shop Employee:** Thank you. And what's the best phone number to reach you?

**Customer:** My number is 555-123-4567.

**Pizza Shop Employee:** Great, your total comes to $24.50. It should take about 30-45 minutes for your delivery. Is there anything else I can help you with?

**Customer:** No, that’s all. Thank you!

**Pizza Shop Employee:** You’re welcome! Enjoy your pizza and have a great day!

# Explanation:
This dialogue exemplifies a typical phone conversation for o